## Sistema Inteligente de Perguntas e Respostas para Documentos PDF

Este notebook apresenta o desenvolvimento de um **sistema inteligente de Perguntas e Respostas aplicado a arquivos com formato PDF**.

A solução foi inspirada e baseada no artigo “How Extract Data from PDF Using LangChain and Mistral” de Jose Chipana, que demonstra o uso de LangChain, embeddings e modelos de linguagem para extrair e consultar informações de PDFs.

Foram utilizados os pdfs da disciplina como dados de entrada para o sistema

Referencias:

[How extract data from PDF using LangChain and Mistral](https://medium.com/@chipanajose/how-extract-data-from-pdf-using-langchain-and-mistral-74b252fd88a0)

[all-mpnet-base-v2 - modelo de embeddings](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)

[guide to evaluate RAG](https://www.evidentlyai.com/llm-guide/rag-evaluation)


[ library for evaluating and testing Large Language Model (LLM) applications](https://docs.ragas.io/en/stable/getstarted/#step-4-run-your-evaluation)




In [ ]:
!pip install -U langchain-core langchain-community langchain-mistralai langchain-text-splitters faiss-cpu ragas pypdf --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from google.colab import files
from langchain_core.runnables import RunnableMap
from operator import itemgetter


In [ ]:
#fazendo upload
uploaded = files.upload()
#pega nomes
pdf_paths = list(uploaded.keys())



Saving 2024-12-evento-ia-na-pesquisa-slides.pdf to 2024-12-evento-ia-na-pesquisa-slides (1).pdf
Saving slides-pln-redes-neurais-para-proc-linguagem-natural.pdf to slides-pln-redes-neurais-para-proc-linguagem-natural (1).pdf
Saving slides-cnn-redes-neurais-convolucionais.pdf to slides-cnn-redes-neurais-convolucionais (1).pdf
Saving ann-slides-redes-neurais-e-deep-learning.pdf to ann-slides-redes-neurais-e-deep-learning (1).pdf


In [ ]:
#lendo os pdfs e transformando em list[document], cada document tem page_content e metada
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    documents.extend(docs)

len(documents)

388

**Divisão do Texto em Chunks**

Antes de alimentar o sistema RAG, o texto original é dividido em partes menores para facilitar a indexação e melhorar a recuperação de informações.

Utilizamos o RecursiveCharacterTextSplitter com os seguintes parâmetros:

- **chunk_size = 1000** caracteres  
- **chunk_overlap = 150** caracteres de sobreposição

Essas configurações ajudam a manter a continuidade sem perder contexto entre os trechos.

In [ ]:
#dividindo o texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)


chunks = text_splitter.split_documents(documents)
len(chunks)

388

**Criação dos Embeddings**

 Cada chunk de texto é transformado em uma representação vetorial, permitindo que o sistema realize buscas semânticas eficientes.


Foi utilizado **sentence-transformers/all-mpnet-base-v2**


Esse modelo é amplamente usado em tarefas de similaridade semântica por apresentar alto desempenho em benchmarks como o STS (Semantic Textual Similarity).

In [ ]:
#criando os embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/tmp/ipython-input-312623789.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Armazenamento dos Embeddings com FAISS**

Para permitir buscas rápidas e eficientes pelos vetores gerados, utilizamos o **FAISS**, uma biblioteca otimizada para operações de similaridade em grandes coleções de embeddings.



In [ ]:
#db eficiente para armazenar embeddings
vector_store = FAISS.from_documents(chunks, embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


**Instanciando  Mistral**

O modelo utilizado no pipeline RAG é o **Mistral Small**, acessado via API.

A instanciação do LLM é feita através do nome do modelo e a da chave de acesso.

 - A chave foi deixada explícita apenas para fins de teste, já que expira em um dia.  


In [ ]:
#instancia modelo, deixei a chave porque ela vence em um dia
llm = ChatMistralAI( model="mistral-small-latest", api_key="key" )

In [ ]:
#testando mistral
resp = llm.invoke("O que é uma rede neural?")
print(resp.content)


Uma **rede neural** é um modelo computacional inspirado no funcionamento do cérebro humano, composto por camadas de **neurônios artificiais** que processam informações. Ela é uma das principais ferramentas da **inteligência artificial (IA)** e do **aprendizado de máquina (machine learning)**.

### **Componentes Básicos:**
1. **Neurônios Artificiais (Nós):**
   - Unidades de processamento que recebem entradas, aplicam pesos e funções de ativação, e geram saídas.
   - Cada neurônio tem:
     - **Entradas (inputs):** Dados de entrada.
     - **Pesos (weights):** Valores que ajustam a importância de cada entrada.
     - **Função de Ativação:** Define como a saída é calculada (ex.: ReLU, sigmoide, tanh).

2. **Camadas:**
   - **Camada de Entrada (Input Layer):** Recebe os dados brutos.
   - **Camadas Ocultas (Hidden Layers):** Processam as informações (podem ter múltiplas camadas em redes profundas, *deep learning*).
   - **Camada de Saída (Output Layer):** Gera o resultado final.

3. **Con

In [ ]:
#criando prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especializado extração e busca de respostas em pdfs/ ."),
    ("human", "Use os trechos abaixo para responder a pergunta:\n\n{context}\n\nPergunta: {question}")
])

In [ ]:
#formantando doc
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

O pipeline abaixo utiliza **RunnableMap** para estruturar o fluxo do RAG.
Ele realiza três etapas principais:

1. **Recuperação de contexto** a partir da pergunta.
2. **Geração da resposta** usando o LLM com o contexto recuperado.
3. **Formatação da saída final** contendo pergunta, resposta e contexto.

In [ ]:


#pipeline que implementa RAG
rag_chain = (
    RunnableMap({
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question"),
    })
    | {
        "answer_output": prompt | llm,
        "question": itemgetter("question"),
        "context": itemgetter("context"),
    }
    | RunnableMap({
        "question": itemgetter("question"),
        "answer": lambda x: x["answer_output"].content,
        "context": itemgetter("context"),
    })
)

**Execução do Pipeline RAG**

Nesta etapa, avaliamos o comportamento do pipeline RAG utilizando o método invoke.A partir de cada questão enviada, o pipeline retorna:

- **Pergunta enviada ao modelo**
- **Resposta gerada pelo LLM**
- **Contextos recuperados pelo retriever**




In [ ]:

resposta = rag_chain.invoke({"question": "Do que falam os slides?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



Pergunta: Do que falam os slides?
Resposta: Os slides abordam os seguintes tópicos:

1. **Função de Ativação** (Slide 28): Provavelmente discute o conceito de funções de ativação em redes neurais, que são usadas para introduzir não-linearidade na rede, permitindo que ela aprenda padrões complexos.

2. **Mecanismo de Atenção** (Slide 62): Likely covers attention mechanisms in neural networks, which are techniques used to allow the network to focus on specific parts of the input data, improving performance on tasks like translation and image captioning.

3. **Detecção de Objetos** (Slide 69): Probably discusses object detection techniques, which involve identifying and locating objects within an image or video.

4. **Informações de Contato e Links** (Slide 36): Provides contact information (denilsonpereira@ufla.br) and links to the presenter's website and the presentation itself.

Os slides parecem ser parte de uma apresentação sobre tópicos avançados em aprendizado de máquina e visão co

In [ ]:
resposta = rag_chain.invoke({"question": "O que é convolução sobre volumes ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: O que é convolução sobre volumes ?
Resposta: Convolução sobre volumes é um processo que envolve a aplicação de um filtro em uma imagem que possui múltiplos canais, como uma imagem colorida que possui os canais RGB (vermelho, verde e azul). Neste contexto, tanto a imagem quanto o filtro são representados como volumes tridimensionais.

Para realizar a convolução, a imagem e o filtro devem ter o mesmo número de canais. Por exemplo, se a imagem tem dimensões 6x6x3 (6 linhas, 6 colunas e 3 canais), o filtro deve ter dimensões 3x3x3 (3 linhas, 3 colunas e 3 canais). O resultado da convolução será uma imagem com dimensões reduzidas, como 4x4, no exemplo dado.

O processo de convolução sobre volumes envolve a multiplicação elemento-a-elemento dos valores dos canais correspondentes da imagem e do filtro, seguida de um somatório desses produtos. Esse procedimento é repetido para cada posição possível do filtro sobre a imagem, resultando em uma nova imagem que destaca certas característ

In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




Pergunta: Qual slide fala sobre DeepFace ?
Resposta: O slide que fala sobre DeepFace é o slide número 65.
Contextos: 65
DeepFace
... ...
Taigman et al., 2014. DeepFace: Closing the Gap to Human-Level Performance in Face Verification
... ...
f (img 1)
f (img 2)
A rede deve aprender os parâmetros de forma que:
   – Se imgi e imgj são a mesma pessoa então d(imgi, imgj) é pequena
   – Se imgi e imgj são pessoas diferentes então d(imgi, imgj) é grande

41
Descida do Gradiente

27
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)

3
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)


In [ ]:
resposta = rag_chain.invoke({"question": "O que fala sobre Extração/Geração de Respostas de um Texto?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




Pergunta: O que fala sobre Extração/Geração de Respostas de um Texto?
Resposta: O trecho sobre Extração/Geração de Respostas de um Texto menciona que essa tarefa envolve extrair ou gerar a resposta para uma pergunta dado um contexto. Essa atividade é parte das principais aplicações de Processamento de Linguagem Natural (PLN), sendo utilizada em sistemas como assistentes virtuais e chatbots. A extração de respostas pode ser extrativa, quando a resposta é retirada diretamente do contexto, ou abstrativa, quando a resposta é gerada a partir do contexto.

Referência: https://www.nlplanet.org/course-practical-nlp/02-practical-nlp-first-tasks/17-question-answering.html
Contextos: 172
Principais Aplicações
Resposta a Pergunta (Question Answering)
 Retorna uma resposta dada uma pergunta
 Assistentes virtuais como Alexa, Siri ou Google
 Dois tipos de tarefas:
 Extrativa: extrai a resposta de um dado contexto
 Abstrativa: gera uma resposta a partir do contexto

174
Principais Aplicações
Sumarizaç

In [ ]:
resposta = rag_chain.invoke({"question": "Qual é a efiencia do Transformer ?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




Pergunta: Qual é a efiencia do Transformer ?
Resposta: Os trechos fornecidos não abordam diretamente a eficiência do Transformer. Eles discutem a arquitetura do modelo, incluindo o encoder e decoder, a codificação posicional e os conceitos de Query, Key e Value. Para responder sobre a eficiência do Transformer, seria necessário informações específicas sobre o desempenho do modelo em termos de tempo de treinamento, consumo de recursos computacionais e precisão em tarefas de processamento de linguagem natural. Se você tiver mais informações ou trechos que abordem esses aspectos, posso ajudar a extrair e resumir as informações relevantes.
Contextos: 88
Transformer: Arquitetura do Modelo
Referência: https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html

89
Transformer: Arquitetura do Modelo
Encoder e Decoder funcionam como discutido 
anteriormente
A arquitetura pode conter várias camadas, usualmente 6
Vamos discutir, a seguir, os outros componentes do 
modelo

95
Codificação P

In [ ]:
resposta = rag_chain.invoke({"question": "Como funciona enconder e decoder?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: Como funciona enconder e decoder?
Resposta: O Encoder-Decoder é uma arquitetura de modelo de aprendizado de máquina usada para tarefas de sequência para sequência, como tradução de texto. Aqui está como funciona:

1. **Encoder**:
   - O Encoder processa a sequência de entrada (por exemplo, uma frase em um idioma).
   - Ele comprime a informação da sequência de entrada em um vetor de contexto de tamanho fixo, chamado \( h_t \).

2. **Decoder**:
   - O Decoder é inicializado com o vetor de contexto \( h_t \) gerado pelo Encoder.
   - Ele usa esse vetor para emitir a saída transformada (por exemplo, a frase traduzida para outro idioma).

### Problemas e Soluções:
- **Problema**: O Encoder-Decoder tradicional não funciona bem para sequências longas porque o vetor de contexto tem de codificar muita informação em um tamanho fixo.
- **Solução**: Adicionar um Mecanismo de Atenção para ajudar a lidar com sequências mais longas e complexas.

### Arquitetura Muitos-pra-Muitos:
- Essa ar

In [ ]:
resposta = rag_chain.invoke({"question": "Qual a arquitetura das RNNS?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



Pergunta: Qual a arquitetura das RNNS?
Resposta: As Redes Neurais Recorrentes (RNNs) possuem uma arquitetura composta pelos seguintes componentes principais:

1. **Entrada**
2. **Estado oculto (hidden state)**: Este estado é responsável por vetorizar o contexto, ou seja, armazenar e processar informações ao longo do tempo.
3. **Pesos (parâmetros)**: São os valores que a rede ajusta durante o treinamento para minimizar erros.
4. **Saída**: O resultado produzido pela rede após processar a entrada e o estado oculto.

Além disso, as RNNs podem ser implementadas em diferentes arquiteturas, como a **Rede Neural Recorrente Bidirecional** e a **LSTM (Long Short Term Memory)**. A LSTM, em particular, é projetada para garantir que o gradiente flua por um longo período de tempo, evitando que ele desapareça (vanish) ou exploda (explode), preservando assim informações contextuais relevantes.
Contextos: 55
Diferentes 
Arquiteturas de RNNs

35
Rede Neural Recorrente Bidirecional
Referência.: https://

In [ ]:
resposta = rag_chain.invoke({"question": "Em qual pagina fala sobre a função de ativação Softmax?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



Pergunta: Em qual pagina fala sobre a função de ativação Softmax?
Resposta: A função de ativação Softmax é mencionada na página 39 do documento.
Contextos: 71
Classificação de Objetos
             Softmax (5)
1 – Gato
2 – Cachorro
3 – Pessoa
4 - Carro
5 – Fundo (sem objetos)
...
...

39
Função Softmax  (ou softargmax)
s

96
Camadas Linear Final e Softmax
Referência: https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html
A saída da pilha do Decoder retorna um vetor 
de floats. Como torná-lo uma palavra?
A camada linear é uma rede densa contendo n 
células, onde n é o tamanho do vocabulário. 
Cada célula corresponde à pontuação (score) 
de uma palavra distinta. Esse vetor de n 
elementos é chamado de logits.
A camada Softmax transforma as pontuações 
em probabilidades. A palavra associada à 
célula com mais alta probabilidade é 
produzida como saída.

IA na Pesquisa
Exemplos de Aplicação
Prof. Denilson Alves Pereira
denilsonpereira@ufla.br
https:/ /sites.google.com/ufla.br/de

In [ ]:
resposta = rag_chain.invoke({"question": "O que é Rede Neural Convolucional?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: O que é Rede Neural Convolucional?
Resposta: Uma Rede Neural Convolucional (CNN) é um tipo específico de rede neural projetada principalmente para processamento de imagens. No entanto, também pode ser utilizada para processamento de linguagem natural e outras aplicações.
Contextos: 37
Matematicamente, visa garantir que o gradiente flua por 
um longo período de tempo e não desapareça (vanish) 
ou exploda (explode)
Possui quatro redes neurais interagindo de forma a 
preservar e compartilhar longas informações contextuais
 RNN possui apenas uma única camada de rede neural
LSTM (Long Short Term Memory)

5
Rede Neural Convolucional (CNN)
Uma CNN (Convolutional Neural Network) é um tipo 
específico de rede neural projetada para 
processamento de imagens
Pode usada também para processamento de 
linguagem natural e outras aplicações

96
denilsonpereira@ufla.br
https://sites.google.com/ufla.br/denilsonpereira/
Link da apresentação:
https:/ /sites.google.com/ufla.br/denilsonpereira/
ho

In [ ]:
resposta = rag_chain.invoke({"question": "Como é arquitetura do GPT?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



Pergunta: Como é arquitetura do GPT?
Resposta: A arquitetura do GPT (Generative Pre-trained Transformer) é baseada em um modelo de transformador (transformer) que utiliza camadas de autoatenção (self-attention) e feed-forward. O GPT é pré-treinado em uma grande quantidade de dados textuais para aprender representações de linguagem, e depois pode ser ajustado para tarefas específicas (fine-tuning) ou usado em contextos de few-shot, one-shot ou zero-shot, dependendo da quantidade de exemplos de treino disponíveis.

Os termos mencionados nos trechos fornecidos referem-se a diferentes aspectos do uso do GPT e de técnicas relacionadas à visão computacional, mas não descrevem diretamente a arquitetura do modelo. A arquitetura do GPT é composta por:

1. **Camadas de Autoatenção (Self-Attention):** Permitem que o modelo considere relações entre todas as palavras em uma sequência, capturando dependências de longo alcance.

2. **Camadas Feed-Forward:** Aplicadas após as camadas de autoatenção pa

#### Avaliação do RAG

In [ ]:
from ragas.metrics import (answer_correctness,faithfulness, context_recall,answer_relevancy)
from ragas import evaluate
from datasets import Dataset



In [ ]:
#pega questions, respostas do rag, groud_truth, contex_retrieved

questions = []
rag_answers = []
ground_truths = []
retrieved_contexts_list = []

for gt_entry in ground_truth_dataset:
    question = gt_entry["question"]
    ground_truth = gt_entry["answer"]

    resposta = rag_chain.invoke({"question": question})
    questions.append(question)
    rag_answers.append(resposta["answer"])
    ground_truths.append(ground_truth)
    contexts_list = resposta["context"].split('\n\n')
    retrieved_contexts_list.append(contexts_list)

In [ ]:



eval_data = {
    "question": questions,
    "answer": rag_answers,
    "ground_truth": ground_truths,
    "contexts": retrieved_contexts_list
}

dataset = Dataset.from_dict(eval_data)



In [ ]:
result = evaluate(
    dataset,
    metrics=[
        answer_correctness,   # equivalente a acurácia
        faithfulness,         # equivalente a precisão
        context_recall,       # equivalente a recall
        answer_relevancy      # equivalente a F1
    ],
    llm=llm,
    embeddings=embeddings
)

print(result)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'answer_correctness': 0.6492, 'faithfulness': 0.7250, 'context_recall': 0.5000, 'answer_relevancy': 0.5995}


### Análise das Métricas

As metricas obtidas na avaliação do sistema RAG utilizando o modelo **Mistral** indicam um desempenho moderado.

**Answer Correctness:** 0.6494

O modelo gera respostas semanticamente coerentes com o ground truth em 68% dos casos. Esse valor é esperado para um modelo de porte médio e um RAG simples.

**Faithfulness:**  0.7250

As respostas estão razoavelmente alinhadas aos trechos recuperados.O  modelo não alucina em excesso, mas ainda pode melhorar.

**Context Recall:**  0.50
Apenas metade das informações relevantes foram recuperadas pelo retriever. Mostra que o retriever é um ponto no RAG que precisa de melhoria.


**Answer Relevancy:** 0.5995
As respostas são moderamente relevantes para a pergunta.



### Comparação experimentos

###



In [ ]:
#troca embedding para experimento
embeddings_base = HuggingFaceEmbeddings(
    model_name="bert-base-uncased"
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#db eficiente para armazenar embeddings
vector_store_experiment = FAISS.from_documents(chunks, embeddings_base)

retriever_experiment = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [ ]:
#pipeline que implementa RAG
rag_chain = (
    RunnableMap({
        "context": itemgetter("question") | retriever_experiment | format_docs,
        "question": itemgetter("question"),
    })
    | {
        "answer_output": prompt | llm,
        "question": itemgetter("question"),
        "context": itemgetter("context"),
    }
    | RunnableMap({
        "question": itemgetter("question"),
        "answer": lambda x: x["answer_output"].content,
        "context": itemgetter("context"),
    })
)

In [ ]:

resposta = rag_chain.invoke({"question": "Do que falam os slides?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: Do que falam os slides?
Resposta: Os slides abordam os seguintes tópicos:

1. **Função de Ativação** (Slide 28): Provavelmente discute o conceito de funções de ativação em redes neurais, que são usadas para introduzir não-linearidade na rede, permitindo que ela aprenda padrões complexos.

2. **Mecanismo de Atenção** (Slide 62): Aborda o mecanismo de atenção, uma técnica usada em processamento de linguagem natural e visão computacional para permitir que o modelo se concentre em partes específicas de entrada que são mais relevantes para a tarefa em questão.

3. **Detecção de Objetos** (Slide 69): Trata da detecção de objetos, um campo da visão computacional que envolve identificar e localizar objetos em imagens ou vídeos.

Além disso, os slides incluem informações de contato (denilsonpereira@ufla.br) e links para o site e apresentação do palestrante.
Contextos: 28
Função de Ativação

62
Mecanismo de 
Atenção

69
Detecção de Objetos

36
denilsonpereira@ufla.br
https://sites.goog

In [ ]:
resposta = rag_chain.invoke({"question": "O que é convolução sobre volumes ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: O que é convolução sobre volumes ?
Resposta: Convolução sobre volumes é um processo matemático aplicado em imagens ou dados que possuem múltiplos canais, como imagens coloridas que possuem três canais (RGB). Neste processo, um filtro (também conhecido como kernel) é aplicado sobre a imagem, realizando um somatório da multiplicação elemento-a-elemento dos canais correspondentes.

Para que a convolução sobre volumes seja possível, o número de canais da imagem e do filtro deve ser o mesmo. Por exemplo, se a imagem possui 3 canais (como uma imagem RGB), o filtro também deve possuir 3 canais.

No exemplo fornecido:
- A imagem tem dimensões 6x6x3 (6 linhas, 6 colunas e 3 canais).
- O filtro tem dimensões 3x3x3 (3 linhas, 3 colunas e 3 canais).
- O resultado da convolução tem dimensões 4x4 (4 linhas e 4 colunas), pois o filtro reduz a dimensão da imagem em 2 pixels em cada direção (6 - 3 + 1 = 4).

O processo de convolução sobre volumes envolve a multiplicação elemento-a-elemento do

In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




Pergunta: Qual slide fala sobre DeepFace ?
Resposta: O slide que fala sobre DeepFace é o slide número 65.
Contextos: 65
DeepFace
... ...
Taigman et al., 2014. DeepFace: Closing the Gap to Human-Level Performance in Face Verification
... ...
f (img 1)
f (img 2)
A rede deve aprender os parâmetros de forma que:
   – Se imgi e imgj são a mesma pessoa então d(imgi, imgj) é pequena
   – Se imgi e imgj são pessoas diferentes então d(imgi, imgj) é grande

41
Descida do Gradiente

27
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)

3
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)


In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

Pergunta: Qual slide fala sobre DeepFace ?
Resposta: O slide que fala sobre DeepFace é o slide número 65.
Contextos: 65
DeepFace
... ...
Taigman et al., 2014. DeepFace: Closing the Gap to Human-Level Performance in Face Verification
... ...
f (img 1)
f (img 2)
A rede deve aprender os parâmetros de forma que:
   – Se imgi e imgj são a mesma pessoa então d(imgi, imgj) é pequena
   – Se imgi e imgj são pessoas diferentes então d(imgi, imgj) é grande

41
Descida do Gradiente

27
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)

3
Aplicações em Processamento de Imagem
 Classificação de imagem
 Detecção de objetos
 Reconhecimento de faces
 Veículos autônomos
Cachorro? (S/N)


In [ ]:
resposta = rag_chain.invoke({"question": "O que fala sobre Extração/Geração de Respostas de um Texto?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: O que fala sobre Extração/Geração de Respostas de um Texto?
Resposta: O trecho sobre Extração/Geração de Respostas de um Texto menciona que essa tarefa envolve extrair ou gerar a resposta para uma pergunta dado um contexto. Essa atividade é parte das principais aplicações de Processamento de Linguagem Natural (PLN), como resposta a perguntas (Question Answering) e sumarização. No caso específico da resposta a perguntas, existem dois tipos de tarefas: extrativa, que extrai a resposta de um dado contexto, e abstrativa, que gera uma resposta a partir do contexto. Essa aplicação é utilizada em assistentes virtuais como Alexa, Siri ou Google. Já na sumarização, a tarefa pode ser extrativa, extraindo a informação relevante, ou abstrativa, gerando um novo texto com a informação relevante.
Contextos: 172
Principais Aplicações
Resposta a Pergunta (Question Answering)
 Retorna uma resposta dada uma pergunta
 Assistentes virtuais como Alexa, Siri ou Google
 Dois tipos de tarefas:
 Extra

In [ ]:

resposta = rag_chain.invoke({"question": "Qual é a efiencia do Transformer ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

Pergunta: Qual é a efiencia do Transformer ?
Resposta: Os trechos fornecidos não mencionam diretamente a eficiência do Transformer. Eles discutem a arquitetura do modelo, incluindo o encoder e decoder, a codificação posicional e os conceitos de Query, Key e Value. Para obter informações sobre a eficiência do Transformer, seria necessário consultar fontes que abordem esse aspecto específico, como estudos de caso, benchmarks ou artigos que comparam o desempenho do Transformer com outros modelos. Se você tiver acesso a esses materiais, posso ajudá-lo a extrair as informações relevantes.
Contextos: 88
Transformer: Arquitetura do Modelo
Referência: https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html

89
Transformer: Arquitetura do Modelo
Encoder e Decoder funcionam como discutido 
anteriormente
A arquitetura pode conter várias camadas, usualmente 6
Vamos discutir, a seguir, os outros componentes do 
modelo

95
Codificação Posicional
Referência: https://luv-bansal.medium.com/t

In [ ]:
resposta = rag_chain.invoke({"question": "Como funciona enconder e decoder?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: Como funciona enconder e decoder?
Resposta: O Encoder-Decoder é uma arquitetura de modelo de aprendizado de máquina usada para tarefas de sequência para sequência, como tradução de texto. Aqui está como funciona:

1. **Encoder**:
   - O Encoder processa a sequência de entrada (por exemplo, uma frase em um idioma) e comprime a informação em um vetor de contexto \( h_t \) de tamanho fixo. Esse vetor de contexto é uma representação compacta da informação contida na sequência de entrada.

2. **Decoder**:
   - O Decoder é inicializado com o vetor de contexto \( h_t \) gerado pelo Encoder e emite a saída transformada (por exemplo, a tradução da frase para outro idioma). O Decoder usa esse vetor de contexto para gerar a sequência de saída.

**Problema e Solução**:
- **Problema**: O modelo Encoder-Decoder básico não funciona bem para sequências longas porque o vetor de contexto tem de codificar muita informação em um tamanho fixo, o que pode levar à perda de informações importantes.


In [ ]:

resposta = rag_chain.invoke({"question": "Qual a arquitetura das RNNS?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

Pergunta: Qual a arquitetura das RNNS?
Resposta: As Redes Neurais Recorrentes (RNNs) possuem uma arquitetura composta por quatro componentes principais:

1. **Entrada**: Recebe os dados de entrada.
2. **Estado oculto (hidden state)**: Atua como a vetorização do contexto, armazenando informações relevantes do passado para influenciar as saídas futuras.
3. **Pesos (parâmetros)**: Parâmetros que são ajustados durante o treinamento para otimizar o desempenho da rede.
4. **Saída**: Produz a saída da rede com base nos dados de entrada e no estado oculto.

Além disso, as RNNs podem ter diferentes arquiteturas, como a **Rede Neural Recorrente Bidirecional** e a **LSTM (Long Short Term Memory)**, que possui quatro redes neurais interagindo para preservar e compartilhar informações contextuais de longo prazo. A LSTM é projetada para garantir que o gradiente flua por um longo período de tempo, evitando que ele desapareça (vanish) ou exploda (explode).
Contextos: 55
Diferentes 
Arquiteturas de RNN

In [ ]:
resposta = rag_chain.invoke({"question": "Em qual pagina fala sobre a função de ativação Softmax?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: Em qual pagina fala sobre a função de ativação Softmax?
Resposta: A função de ativação Softmax é mencionada na página 39 do documento.
Contextos: 71
Classificação de Objetos
             Softmax (5)
1 – Gato
2 – Cachorro
3 – Pessoa
4 - Carro
5 – Fundo (sem objetos)
...
...

39
Função Softmax  (ou softargmax)
s

96
Camadas Linear Final e Softmax
Referência: https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html
A saída da pilha do Decoder retorna um vetor 
de floats. Como torná-lo uma palavra?
A camada linear é uma rede densa contendo n 
células, onde n é o tamanho do vocabulário. 
Cada célula corresponde à pontuação (score) 
de uma palavra distinta. Esse vetor de n 
elementos é chamado de logits.
A camada Softmax transforma as pontuações 
em probabilidades. A palavra associada à 
célula com mais alta probabilidade é 
produzida como saída.

IA na Pesquisa
Exemplos de Aplicação
Prof. Denilson Alves Pereira
denilsonpereira@ufla.br
https:/ /sites.google.com/ufla.br/de

In [ ]:

resposta = rag_chain.invoke({"question": "O que é Rede Neural Convolucional?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


Pergunta: O que é Rede Neural Convolucional?
Resposta: Uma Rede Neural Convolucional (CNN) é um tipo específico de rede neural projetada principalmente para processamento de imagens. No entanto, também pode ser utilizada para outras aplicações, como processamento de linguagem natural.
Contextos: 37
Matematicamente, visa garantir que o gradiente flua por 
um longo período de tempo e não desapareça (vanish) 
ou exploda (explode)
Possui quatro redes neurais interagindo de forma a 
preservar e compartilhar longas informações contextuais
 RNN possui apenas uma única camada de rede neural
LSTM (Long Short Term Memory)

5
Rede Neural Convolucional (CNN)
Uma CNN (Convolutional Neural Network) é um tipo 
específico de rede neural projetada para 
processamento de imagens
Pode usada também para processamento de 
linguagem natural e outras aplicações

96
denilsonpereira@ufla.br
https://sites.google.com/ufla.br/denilsonpereira/
Link da apresentação:
https:/ /sites.google.com/ufla.br/denilsonpereira

In [ ]:
resposta = rag_chain.invoke({"question": "Como é arquitetura do GPT?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

Pergunta: Como é arquitetura do GPT?
Resposta: A arquitetura do GPT (Generative Pre-trained Transformer) é baseada em um modelo de transformador, que é um tipo de rede neural projetada para lidar com dados sequenciais, como texto. Aqui estão os principais componentes e conceitos relacionados à arquitetura do GPT:

1. **Fine-tuning**: Este é um processo de ajuste fino do modelo usando muitos exemplos de treino para atualizar os pesos da rede neural. Isso permite que o modelo se adapte melhor a tarefas específicas.

2. **Few-shot**: Neste método, poucos exemplos de treino são usados para condicionar o modelo, mas não há atualização dos pesos. Isso significa que o modelo usa esses exemplos para melhorar seu desempenho em tarefas específicas sem alterar sua estrutura interna.

3. **One-shot**: Similar ao few-shot, mas com apenas um exemplo de treino. O modelo usa esse único exemplo para condicionar seu comportamento, sem atualizar os pesos.

4. **Zero-shot**: Neste caso, o modelo não receb

### Avaliando outra config

In [ ]:
#pega questions, respostas do rag, groud_truth, contex_retrieved

questions = []
rag_answers = []
ground_truths = []
retrieved_contexts_list = []

for gt_entry in ground_truth_dataset:
    question = gt_entry["question"]
    ground_truth = gt_entry["answer"]

    resposta = rag_chain.invoke({"question": question})
    questions.append(question)
    rag_answers.append(resposta["answer"])
    ground_truths.append(ground_truth)
    contexts_list = resposta["context"].split('\n\n')
    retrieved_contexts_list.append(contexts_list)

In [ ]:
eval_data_experiment = {
    "question": questions,
    "answer": rag_answers,
    "ground_truth": ground_truths,
    "contexts": retrieved_contexts_list
}

dataset = Dataset.from_dict(eval_data_experiment)



In [ ]:

result = evaluate(
    dataset,
    metrics=[
        answer_correctness,   # equivalente a acurácia
        faithfulness,         # equivalente a precisão
        context_recall,       # equivalente a recall
        answer_relevancy      # equivalente a F1
    ],
    llm=llm,
    embeddings=embeddings
)

print(result)


Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]

{'answer_correctness': 0.6063, 'faithfulness': 0.8400, 'context_recall': 0.5000, 'answer_relevancy': 0.5667}


**Análise das Métricas**



Foi feita uma execução do experimento RAG, com mudança de embeddings para o modelo de embeddings
[bert-base-uncased](https://huggingface.co/google-bert/bert-base-uncased).




As métricas obtidas na nova avaliação do sistema RAG utilizando o modelo Mistral mostram uma performance moderada, com algumas melhorias e outras quedas em relação à execução anterior.



**Answer Correctness:** 0.6063

O modelo produziu respostas semanticamente compatíveis com o ground truth em cerca de 61% dos casos.
O desempenho caiu um pouco em relação ao experimento anterior, devido à mudança nos embeddings ou ao conjunto de chunks recuperados. Mesmo assim o resultado continua dentro do esperado para um pipeline RAG simples.

**Faithfulness:** 0.8400

O aumento de  Faithfulness sugere que o embedding atual ajudou o modelo a usar melhor o contexto que recebe. Foi a melhor métrica desse experimento.

**Context Recall:** 0.50

Assim como antes, apenas 50% das informações realmente relevantes foram recuperadas.


**Answer Relevancy**: 0.5667

O valor aumentou em relação ao experimento anterior (0.36 para 0.56), indicando que: as respostas agora estão mais conectadas às perguntas, o modelo está interpretando melhor o que o usuário deseja, e o novo embedding contribuiu para fornecer contexto mais alinhado as consultas.



### Comparação entre os dois modelos de embeddings e justificativa dos resultados


A comparação entre os dois experimentos mostra que a escolha do embedding impacta diretamente o desempenho do RAG.
O **all-mpnet-base-v2** apresentou maior answer correctness, indicando respostas mais próximas ao ground truth, mas teve menor faithfulness e answer relevancy, sugerindo que o contexto recuperado nem sempre era adequado.

No **bert-base-uncased**, houve melhora clara em faithfulness e answer relevancy, mostrando que o modelo passou a usar melhor o contexto e a alucinar menos, embora a correção total tenha caido um pouco.

Em ambos os casos, o context recall permaneceu em 0.50, indicando que o principal limite do sistema está na recuperação de documentos, não no LLM ou no embedding.